General program for SQS derivation. Take $P6/mmm$ lattice as example.

In [1]:
using Random
using LinearAlgebra
using NPZ
using Combinatorics
using Statistics
using JLD2
using Distributed
using JSON
using Base.Threads
using IJulia
using Profile
using BenchmarkTools
using .GC
using PyPlot

function zero_clip(num, x)
    if x == 0
        return 0
    end
    
    return num
end

println(Threads.nthreads())
pth_loadbase = ""
@everywhere include("$(pth_loadbase)cpr_rspace.jl")
@everywhere include("$(pth_loadbase)basis_func_quin.jl")
@everywhere include("$(pth_loadbase)cpr_sqs.jl")

8


In [3]:
atom_num = 64 #TODO number of atoms
pth_loadbase = ""
basis_norm_dict_pth = "$(pth_loadbase)utils/basis_norm_dict_jl_quin.json" 

ind_cluster_dict_pth = "$(pth_loadbase)utils/1100/tm_4_4_2_64/cluster_ind_4_4_2_jl.json"
basis_norm_dict = JSON.parsefile(basis_norm_dict_pth)
ind_cluster_dict = JSON.parsefile(ind_cluster_dict_pth)

symbol_list_pth = "$(pth_loadbase)utils/symbol_list_240111_6nn.json"
symbol_list = JSON.parsefile(symbol_list_pth)
println(length(symbol_list))

config_info = config(
        ind_cluster_dict,
        symbol_list,
        basis_norm_dict,
        # cluster_norm_dict,
        true, true, true, true)

6


config(Dict{Any, Any}("4" => Any[Any[1, 3], Any[1, 34], Any[1, 308], Any[1, 370], Any[1, 835], Any[1, 868], Any[2, 4], Any[2, 17], Any[2, 19], Any[2, 353]  …  Any[62, 896], Any[62, 1421], Any[62, 1423], Any[63, 957], Any[63, 1438], Any[63, 1440], Any[64, 925], Any[64, 958], Any[64, 1423], Any[64, 1485]], "1" => Any[Any[1, 2], Any[1, 17], Any[1, 18], Any[1, 369], Any[1, 836], Any[1, 852], Any[2, 3], Any[2, 18], Any[2, 369], Any[2, 370]  …  Any[61, 1422], Any[62, 63], Any[62, 1422], Any[63, 64], Any[63, 1422], Any[63, 1423], Any[63, 1424], Any[64, 941], Any[64, 957], Any[64, 1424]], "5" => Any[Any[1, 19], Any[1, 33], Any[1, 307], Any[1, 353], Any[1, 371], Any[1, 851], Any[2, 20], Any[2, 34], Any[2, 308], Any[2, 354]  …  Any[62, 1424], Any[62, 1438], Any[63, 941], Any[63, 1421], Any[63, 1439], Any[63, 1485], Any[64, 942], Any[64, 1422], Any[64, 1440], Any[64, 1486]], "2" => Any[Any[1, 5], Any[1, 717], Any[2, 6], Any[2, 718], Any[3, 7], Any[3, 719], Any[4, 8], Any[4, 720], Any[5, 9], Any[6

Generate SQS

##### Single-composition

In [ ]:
function spin_exchange(sigma, action)
    sigma_n = copy(sigma)
    sigma_n[action[1]], sigma_n[action[2]] = sigma[action[2]], sigma[action[1]]
    return sigma_n
end

compo_list = Matrix{Float64}([
    0 0.3 0.4 0.3 0;
])

compo_denote = join(convert(Vector{Int64}, round.(compo_list[1,:]*100),), "")
#TODO Set the path
system_denote = "ter_Cij"
sav_pth = "sqs_zoo"
meta_iter = 12
atom_num = 64 #TODO number of transition metal atom
lagrange_list = LinRange(1e-5, 1e-3, meta_iter)

# Threads.@threads for meta_i in 1:meta_iter
for meta_i in 1:meta_iter

    #TODO specify the MC steps
    mc_iter = 10000
    lagrange = lagrange_list[meta_i]
    # lagrange = 1e-5
    temp = 2
    res_min = Inf
    ele_list_min, cprvec_min = nothing, nothing
    res_list = []
    global res_list
    cprvec_sqs = main_sqs(config_info, compo_list)[1] #TODO as the ground truth

    #TODO rule for sequence generation
    ele_list = ele_list_gen(compo_list[1,:], atom_num, 1, "randchoice")
    cpr_sigma = main_rspace(1:1, config_info, ele_list)[1]
    res_raw = norm(cpr_sigma.-cprvec_sqs)

    for i in 1:mc_iter 
        temp = exp(-i/100)
        action = rand(1:atom_num, 2)
        ele_list_n = spin_exchange(ele_list, action)
        cpr_sigma_n = main_rspace(1:1, config_info, ele_list_n)[1]
        res_n = norm(cpr_sigma_n.-cprvec_sqs)
        penalize = maximum(abs.(cpr_sigma_n.-cprvec_sqs))*lagrange
        res_n += penalize 
        append!(res_list, res_n)

        if minimum([1, exp((res_raw - res_n)/temp)]) > rand()
            ele_list = copy(ele_list_n)
            res_raw = copy(res_n)
            if res_n < res_min
                res_min = copy(res_n)
                cprvec_min = copy(cpr_sigma_n)
                ele_list_min = copy(ele_list)
            end
        end
    end
    npzwrite(joinpath(sav_pth, "sqs_$(system_denote)_$(meta_i)_$(atom_num)_$(compo_denote).npy"), ele_list_min)
end